In [28]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras import backend as K
import pandas as pd
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 

In [ ]:
image_width = 224
image_height = 224

# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
nb_train_samples = 1200 # number of all training samples belong to all possible classses
nb_validation_samples = 198 # number of all validation samples belong to all possible classes
np_test_samples = 198 # number of all test samples belong to all possible classes
#train_dir = "D:/retinal_data_set_original/_1000_per_class_train/augmented_224/train"
train_dir = "D:/retinal_data_set_original/original/new_train"
#validation_dir = "D:/retinal_data_set_original/_1000_per_class_train/augmented_224/validation"
validation_dir = "D:/retinal_data_set_original/original/new_valid"
test_dir = "D:/retinal_data_set_original/original/new_test"

In [20]:
# Returns a compiled model identical to the previous one
loaded_model = load_model('D:/retinal_data_set_original/original/models/vgg16/vgg16_dr_last2.h5')

In [21]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               6422784   
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 1285      
Total params: 21,139,781
Trainable params: 6,424,581
Non-trainable params: 14,715,200
________________________________________________________

In [22]:
intermediate_layer_model = Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer("dense_13").output)

In [23]:
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               6422784   
Total params: 21,137,472
Trainable params: 6,422,784
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#datagen = ImageDataGenerator(rescale=1. / 255)
train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)

# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = intermediate_layer_model.predict_generator(train_generator, nb_train_samples // batch_size)
# save the output as a Numpy array
np.save(open('D:/retinal_data_set_original/original/models/vgg16/bottleneck_features_train_vgg16.npy', 'wb'), bottleneck_features_train)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = intermediate_layer_model.predict_generator(validation_generator, nb_validation_samples // batch_size)
np.save(open('D:/retinal_data_set_original/original/models/vgg16/bottleneck_features_validation_vgg16.npy', 'wb'), bottleneck_features_validation)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_test = intermediate_layer_model.predict_generator(test_generator, nb_test_samples // batch_size)
np.save(open('D:/retinal_data_set_original/original/models/vgg16/bottleneck_features_test_vgg16.npy', 'wb'), bottleneck_features_test)

In [ ]:
train_data = np.load(open('D:/retinal_data_set_original/original/models/vgg16/bottleneck_features_train_vgg16.npy', 'rb'))
# the features were saved in order, so recreating the labels is easy

train_labels = np.array([0] * 600 + [1] * 600 + [2] * 600 + [3] * 600 + [4] * 600)

validation_data = np.load(open('D:/retinal_data_set_original/original/models/vgg16/bottleneck_features_validation_vgg16.npy', 'rb'))
validation_labels = np.array([0] * 96 + [1] * 96 + [2] * 96 + [3] * 96 + [4] * 96)

In [ ]:
# selecting n_estimators
error_results = []
n_estimator_options = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
for trees in n_estimator_options:
    model = RFC(trees, random_state=1, class_weight="balanced", oob_score=True)
    model.fit(train_data, train_labels)
    #predict = model.predict(validation_data)
    err = 1 - model.score(validation_data,validation_labels)
    error_results.append(error)

In [ ]:
#return value in n_estimator_options according to the minimum value index in error_results
min_index = error_results.index(min(error_results))
num_of_trees = n_estimator_options[min_index]

rf = RFC(n_estimators=num_of_trees,max_features="sqrt",max_depth=3,min_samples_leaf=5,criterion="gini",class_weight="balanced")
rf.fit(train_data, train_labels)

In [ ]:
rf_predictions = rf.predict(validation_data)
print (metrics.classification_report(validation_labels, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(validation_labels, rf_predictions), 2))

#cross table for test sample
preds = rf.predict_proba(validation_data)[:,1]
prediction = rf.predict(validation_data)
print (pd.crosstab(index=validation_labels, columns=prediction, rownames=['actual'], colnames=['preds']))